Gossiping-QA-Dataset : https://www.kaggle.com/zake7749/pttgossipingcorpus

In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('Gossiping-QA-Dataset-2_0.csv')

In [3]:
df.head()

,question,answer
0,為什麼 聖結石 會被酸而 這群人 不會？,質感 劇本 成員 都差很多好嗎 不要拿腎結石來污辱這群人
1,為什麼慶祝228會被罵可是慶端午不會？,因為屈原不是台灣人，是楚國人。
2,有沒有戰神阿瑞斯的八卦?,爵士就是阿瑞斯 男主角最後死了
3,理論與實務最脫節的系,哪個系不脫節...你問最不脫節的簡單多了...
4,為什麼PTT這麼多人看棒球,肥宅才看棒球 系壘一堆胖子


In [4]:
print(len(df))
df = df[df['question'].notnull()]
df = df[df['answer'].notnull()]
df = df[df['question'].notna()]
df = df[df['answer'].notna()]
print(len(df))

774114
774108


In [5]:
df['question'][0]

'為什麼 聖結石 會被酸而 這群人 不會？'

In [6]:
df['answer'][0]

'質感 劇本 成員 都差很多好嗎 不要拿腎結石來污辱這群人'

In [7]:
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import BertTokenizer, BertModel

import numpy as np
import random
import math

In [8]:
g_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
g_tokenizer = BertTokenizer.from_pretrained("bert-base-chinese") #'bert-base-uncased'
g_vocab_size = g_tokenizer.vocab_size
g_max_input_length = g_tokenizer.max_model_input_sizes['bert-base-chinese']  # 512
g_bert = BertModel.from_pretrained('bert-base-chinese')
g_bert_emb_dim = g_bert.config.to_dict()['hidden_size']

In [10]:
def tokenize_and_cut(sentence):
    tokens = g_tokenizer.tokenize(sentence)
    tokens = tokens[:g_max_input_length-2]
    return tokens

In [11]:
g_vocab_size

21128

In [12]:
class QADataset(Dataset):
    def __init__(self, train_Q, train_A):
        
        self.train_Q = train_Q
        self.train_A = train_A
        self.length = len(train_Q)
        
    def __getitem__(self, index):
           
#         Q_token = tokenize_and_cut("[CLS] " + str(self.train_Q[index]) + " [SEP]")
#         A_token = tokenize_and_cut("[CLS] " + str(self.train_A[index]) + " [SEP]")

        Q_token = tokenize_and_cut(str(self.train_Q[index]))
        A_token = tokenize_and_cut(str(self.train_A[index]+"。"))
        
        Q_index = g_tokenizer.convert_tokens_to_ids(Q_token)
        A_index = g_tokenizer.convert_tokens_to_ids(A_token)
        
        Q_tensor = torch.tensor([Q_index]).transpose(0, 1).to(g_device)
        A_tensor = torch.tensor([A_index]).transpose(0, 1).to(g_device)
        
        return Q_tensor, A_tensor

    def __len__(self):

        return self.length

In [13]:
train_set = QADataset(df['question'], df['answer'])

In [14]:
train_set.__getitem__(0)

(tensor([[4158],
         [ 784],
         [7938],
         [5469],
         [5178],
         [4767],
         [3298],
         [6158],
         [7000],
         [5445],
         [6857],
         [5408],
         [ 782],
         [ 679],
         [3298],
         [8043]], device='cuda:0'),
 tensor([[6549],
         [2697],
         [1206],
         [3315],
         [2768],
         [1519],
         [6963],
         [2345],
         [2523],
         [1914],
         [1962],
         [1621],
         [ 679],
         [6206],
         [2897],
         [5575],
         [5178],
         [4767],
         [ 889],
         [3738],
         [6802],
         [6857],
         [5408],
         [ 782],
         [ 511]], device='cuda:0'))

In [15]:
def create_mini_batch(samples):

    Q_tensor = [s[0] for s in samples]
    A_tensor = [s[1] for s in samples]
    
    Q_tensor = pad_sequence(Q_tensor)
    A_tensor = pad_sequence(A_tensor)
    
#     masks_Q_tensors = torch.zeros(Q_tensor.size(), dtype=torch.long).to(g_device)
#     masks_Q_tensors = masks_Q_tensors.masked_fill(Q_tensor != 0, 1)
    
#     masks_A_tensors = torch.zeros(A_tensor.size(), dtype=torch.long).to(g_device)
#     masks_A_tensors = masks_A_tensors.masked_fill(A_tensor != 0, 1)
    
#     return Q_tensor, A_tensor, masks_Q_tensors, masks_A_tensors
    return Q_tensor, A_tensor

In [16]:
len(df)

774108

In [17]:
temp = 50000
t_set, v_set = torch.utils.data.random_split(train_set, [temp, len(df)-temp])

model: https://github.com/demi6od/ChatBot/blob/master/image/ChatBotBertTransformer.jpg

In [18]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [19]:
class TransBertEncoder(nn.Module):
    def __init__(self, nhead=8, nlayers=6, dropout=0.5):
        super().__init__()

        # bert encoder
        self.bert = g_bert

        # transformer encoder, as bert last layer fine-tune
        self.pos_encoder = PositionalEncoding(g_bert_emb_dim, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model=g_bert_emb_dim, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)

    def forward(self, src):
        # src = [src len, batch size]

        with torch.no_grad():
            # embedded = [src len, batch size, emb dim]
            embedded = self.bert(src.transpose(0, 1))[0].transpose(0, 1)

        embedded = self.pos_encoder(embedded)
        src_mask = nn.Transformer().generate_square_subsequent_mask(len(embedded)).to(g_device)

        # outputs = [src len, batch size, hid dim * n directions]
        outputs = self.transformer_encoder(embedded, mask = src_mask)

        return outputs

In [20]:
class TransBertDecoder(nn.Module):
    def __init__(self, nhead=8, nlayers=6, dropout=0.5):
        super().__init__()

        self.bert = g_bert

        self.pos_decoder = PositionalEncoding(g_bert_emb_dim, dropout)
        decoder_layer = nn.TransformerDecoderLayer(d_model=g_bert_emb_dim, nhead=nhead)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=nlayers)

        self.fc_out = nn.Linear(g_bert_emb_dim, g_vocab_size)

    def forward(self, tgt, meaning, teacher_forcing_ratio = 0.01):
        # tgt = [output_len, batch size]
        output_len = tgt.size(0)
        batch_size = tgt.size(1)
        
        # teacher_forcing_ratio = 1 -> always true
        teacher_force = random.random() < teacher_forcing_ratio 
        
        if teacher_force and self.training:
            tgt_emb_total = torch.zeros(output_len, batch_size, g_bert_emb_dim).to(g_device)

            for t in range(0, output_len):
                with torch.no_grad():
                    tgt_emb = self.bert(tgt[:t+1].transpose(0, 1))[0].transpose(0, 1)
                        
                tgt_emb_total[t] = tgt_emb[-1]

            tgt_mask = nn.Transformer().generate_square_subsequent_mask(len(tgt_emb_total)).to(g_device)
            decoder_output = self.transformer_decoder(tgt=tgt_emb_total,
                                                      memory=meaning,
                                                      tgt_mask=tgt_mask)
            predictions = self.fc_out(decoder_output)
        else:
            # initialized the input of the decoder with sos_idx (start of sentence token idx)
            output = torch.full((output_len+1, batch_size), g_tokenizer.cls_token_id, dtype=torch.long, device=g_device)
            predictions = torch.zeros(output_len, batch_size, g_vocab_size).to(g_device)

            for t in range(0, output_len):
                with torch.no_grad():
                    # tgt_emb = [t, batch size, emb dim]
                    tgt_emb = self.bert(tgt[:t+1].transpose(0, 1))[0].transpose(0, 1)
                            
                # tgt_emb = self.pos_encoder(tgt_emb)
                tgt_mask = nn.Transformer().generate_square_subsequent_mask(len(tgt_emb)).to(g_device)

                # decoder_output = [t, batch size, emb dim]
                decoder_output = self.transformer_decoder(tgt=tgt_emb,
                                                          memory=meaning,
                                                          tgt_mask=tgt_mask)

                # prediction = [batch size, vocab size]
                prediction = self.fc_out(decoder_output[-1])

                # predictions = [output_len, batch size, vocab size]
                predictions[t] = prediction

                one_hot_idx = prediction.argmax(1)

                # output  = [output len, batch size]
                output[t+1] = one_hot_idx

        return predictions

In [21]:
class Seq2Seq(nn.Module):
    def __init__(self, transbert_encoder, transbert_decoder):
        super().__init__()

        self.transbert_encoder = transbert_encoder
        self.transbert_decoder = transbert_decoder

    def forward(self, src, tgt, teacher_forcing_ratio):
        request_embeddings = self.transbert_encoder(src)
        response_meaning = request_embeddings   
        response = self.transbert_decoder(tgt, response_meaning, teacher_forcing_ratio)
        return response

In [22]:
def print_chat(sentences):
    print("chatbot: ", end="")
    for word_embeds in sentences:
        #pick first
        word_embed = word_embeds[0]
        # find one shot index from word embedding
        max_idx_t = word_embed.argmax()
        max_idx = max_idx_t.item()
        word = g_tokenizer.convert_ids_to_tokens(max_idx)
        print(word, end=" ")
    print("")  # new line at the end of sentence


def print_target_tensor(sentences):
    print("target: ", end="")
    for word_embeds in sentences:
        #pick first
        word_embed = word_embeds[0]
        max_idx = word_embed.item()
        word = g_tokenizer.convert_ids_to_tokens(max_idx)
        print(word, end=" ")
    print("")  # new line at the end of sentence
    
def print_input_tensor(sentences):
    print("input: ", end="")
    for word_embeds in sentences:
        #pick first
        word_embed = word_embeds[0]
        max_idx = word_embed.item()
        word = g_tokenizer.convert_ids_to_tokens(max_idx)
        print(word, end=" ")
    print("")  # new line at the end of sentence

In [23]:
INPUT_DIM = g_vocab_size
OUTPUT_DIM = g_vocab_size
ENC_EMB_DIM = g_bert_emb_dim
DEC_EMB_DIM = g_bert_emb_dim
HID_DIM = 2048  # 5 * 200
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
TRANSFORMER_ENCODER_LAYER = 1  # bert fine-tune
TRANSFORMER_DECODER_LAYER = 3  # semantics -> morphology -> syntax
TRANSFORMER_HEAD = 4

transbert_encoder = TransBertEncoder(TRANSFORMER_HEAD, TRANSFORMER_ENCODER_LAYER, ENC_DROPOUT)
transbert_decoder = TransBertDecoder(TRANSFORMER_HEAD, TRANSFORMER_DECODER_LAYER, DEC_DROPOUT)

In [24]:
g_model = Seq2Seq(transbert_encoder, transbert_decoder).to(g_device)

In [25]:
g_model

Seq2Seq(
  (transbert_encoder): TransBertEncoder(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(21128, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
    

In [26]:
import torch.optim as optim

In [27]:
optimizer = optim.Adam(g_model.parameters())
loss_fn = nn.CrossEntropyLoss(ignore_index=g_tokenizer.pad_token_id)

In [33]:
train_data = DataLoader(t_set, shuffle=True, batch_size=30, collate_fn=create_mini_batch)

In [34]:
len(df)

774108

In [35]:
len(train_data)

1667

In [36]:
iter = len(train_data)

In [37]:
TRAIN_DIALOG = True
for e in range(10): #lucky number
    
    mean_loss = 0
    count = 0
    
    for i, t in train_data:
        
        try:

            optimizer.zero_grad()

            i= i.view(i.shape[0],i.shape[1])
            t = t.view(t.shape[0],t.shape[1])
            
            # src = [src len, batch size]
            # tgt = [output_len, batch size]

            output =  g_model(i, t, 0.5)
            output_dim = output.shape[-1]
            loss = loss_fn(output.view(-1, output_dim), t.view(-1))
            
#             print(output.size())
#             print(output_dim)
#             print(output.view(-1, output_dim).size())
#             print(t.view(-1).size())
            
            loss.backward()
            optimizer.step()

            count += 1
            mean_loss+=loss
            
        except RuntimeError: 
            print('error.....1')
        except AttributeError: 
            print('error.....2')
        else:
            print("e : " + str(e) + ' (' + str(count) + '/' + str(iter) + ') loss : ' + str(float(loss)) )
            if count % 10 == 0:
                print_input_tensor(i)
                print_target_tensor(t)
                print_chat(output)

    print("*****e : "+str(e)+" L : "+str(mean_loss/count)+"*****")
        
    torch.save(g_model, "./transformer_"+str(e)+".pkl")

e : 0 (1/1667) loss : 7.593169212341309
e : 0 (2/1667) loss : 7.22128438949585
e : 0 (3/1667) loss : 7.56218957901001
e : 0 (4/1667) loss : 7.060490608215332
e : 0 (5/1667) loss : 7.233905792236328
e : 0 (6/1667) loss : 6.9640679359436035
e : 0 (7/1667) loss : 6.8874359130859375
e : 0 (8/1667) loss : 6.888753414154053
e : 0 (9/1667) loss : 7.158417224884033
e : 0 (10/1667) loss : 7.065744400024414
input: 等 登 ～ 等 登 ～ 等 登 等 登 等 登 登 登 ～～～ [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 日 本 節 目 創 意 的 ? 評 分 的 旋 律 ? 變 變 變 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 
e : 0 (11/1667) loss : 6.721405982971191
e : 0 (12/1667) loss : 6.975919723510742
e : 0 (13/1667) loss : 6.748508930206299
e : 0 (14/1667) loss : 6.6305742263793945
e : 0 (15/1667) loss : 6.6546196937561035
e : 0 (16/1667) loss : 6.7155046463012695
e : 0 (17/1667) loss : 6.75689172744751
e : 0 (18/1667) loss : 6.661960124969482
e : 0 (19/1667) loss : 6.891993045806885
e : 0 (20/1

e : 0 (131/1667) loss : 6.266629219055176
e : 0 (132/1667) loss : 6.2612624168396
e : 0 (133/1667) loss : 6.212732791900635
e : 0 (134/1667) loss : 6.402521133422852
e : 0 (135/1667) loss : 6.451454162597656
e : 0 (136/1667) loss : 6.691073894500732
e : 0 (137/1667) loss : 6.456526756286621
e : 0 (138/1667) loss : 6.329440593719482
e : 0 (139/1667) loss : 6.359493732452393
e : 0 (140/1667) loss : 6.454122543334961
input: 隊 友 尾 刀 沒 搶 到 大 喊 作 弊 是 什 麼 心 態 ? [PAD] [PAD] [PAD] [PAD] 
target: 謝 謝 美 國 網 友 關 心 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 
e : 0 (141/1667) loss : 6.34480094909668
e : 0 (142/1667) loss : 6.3909454345703125
e : 0 (143/1667) loss : 6.37630558013916
e : 0 (144/1667) loss : 6.867636680603027
e : 0 (145/1667) loss : 6.44823694229126
e : 0 (146/1667) loss : 6.466745853424072
e : 0 (147/1667) loss : 6.371824741363525
e : 0 (148/1667) loss : 6.226798534393310

e : 0 (259/1667) loss : 6.390468597412109
e : 0 (260/1667) loss : 6.513803482055664
input: 整 個 宇 宙 換 算 成 一 顆 紅 豆 的 重 量 [PAD] [PAD] [PAD] [PAD] 
target: 我 比 較 喜 歡 小 宇 宙 因 為 可 以 燃 燒 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 
e : 0 (261/1667) loss : 6.333317756652832
e : 0 (262/1667) loss : 6.527466297149658
e : 0 (263/1667) loss : 6.228643417358398
e : 0 (264/1667) loss : 6.395301818847656
e : 0 (265/1667) loss : 6.2142863273620605
e : 0 (266/1667) loss : 6.427307605743408
e : 0 (267/1667) loss : 6.468784809112549
e : 0 (268/1667) loss : 6.453034400939941
e : 0 (269/1667) loss : 6.530745029449463
e : 0 (270/1667) loss : 6.352621555328369
input: 有 沒 有 怪 我 咯 這 首 歌 的 八 卦 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 放 心 ， 我 不 會 點 進 去 看 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 
e : 0 (271/1667) loss : 6.582279682159424
e

e : 0 (383/1667) loss : 6.196511268615723
e : 0 (384/1667) loss : 6.499212741851807
e : 0 (385/1667) loss : 6.1282501220703125
e : 0 (386/1667) loss : 6.423578262329102
e : 0 (387/1667) loss : 6.169256210327148
e : 0 (388/1667) loss : 6.648978233337402
e : 0 (389/1667) loss : 6.449077129364014
e : 0 (390/1667) loss : 6.615635871887207
input: 快 速 通 道 反 而 更 慢 是 怎 樣 ？ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 你 可 以 走 普 通 驗 票 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 
e : 0 (391/1667) loss : 6.2082014083862305
e : 0 (392/1667) loss : 6.331165313720703
e : 0 (393/1667) loss : 6.433135986328125
e : 0 (394/1667) loss : 6.3952460289001465
e : 0 (395/1667) loss : 6.332516670227051
e : 0 (396/1667) loss : 6.211456775665283
e : 0 (397/1667) loss : 6.488963603973389
e : 0 (398/1667) loss : 6.32354736328125
e : 0 (399/1667) loss : 6.287484169006348
e : 0 (400/1667) loss : 6.345325946807861
input: 學 妹 下 課 要 去 

e : 0 (511/1667) loss : 6.440386772155762
e : 0 (512/1667) loss : 6.3630523681640625
e : 0 (513/1667) loss : 6.223169326782227
e : 0 (514/1667) loss : 6.192883491516113
e : 0 (515/1667) loss : 6.470056056976318
e : 0 (516/1667) loss : 6.496650218963623
e : 0 (517/1667) loss : 6.311891078948975
e : 0 (518/1667) loss : 6.363791465759277
e : 0 (519/1667) loss : 6.19018030166626
e : 0 (520/1667) loss : 6.529807090759277
input: 有 沒 有 氫 氟 酸 的 八 卦 ？ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 上 面 的 [UNK] 蝕 刻 玻 璃 啊 . . . . 你 的 國 中 理 化 老 師 在 哭 。 
chatbot: 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 
e : 0 (521/1667) loss : 6.319301605224609
e : 0 (522/1667) loss : 6.310341835021973
e : 0 (523/1667) loss : 6.396168231964111
e : 0 (524/1667) loss : 6.311518669128418
e : 0 (525/1667) loss : 6.2435078620910645
e : 0 (526/1667) loss : 6.409302234649658
e : 0 (527/1667) loss : 6.2702484130859375
e : 0 (528/1667) loss : 6.253353595733643
e : 0 (529/1667) loss : 6.283752918

...

e : 5 (1336/1667) loss : 6.357649803161621
e : 5 (1337/1667) loss : 6.332252025604248
e : 5 (1338/1667) loss : 6.362033367156982
e : 5 (1339/1667) loss : 6.326687812805176
e : 5 (1340/1667) loss : 6.251494884490967
input: 朋 友 用 鍋 子 在 打 架 有 卦 ？ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 好 難 直 接 公 布 答 案 好 嗎 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 
e : 5 (1341/1667) loss : 6.220963954925537
e : 5 (1342/1667) loss : 6.335628986358643
e : 5 (1343/1667) loss : 6.4471025466918945
e : 5 (1344/1667) loss : 6.268309593200684
e : 5 (1345/1667) loss : 6.34676456451416
e : 5 (1346/1667) loss : 6.222686767578125
e : 5 (1347/1667) loss : 6.300711154937744
e : 5 (1348/1667) loss : 6.168684005737305
e : 5 (1349/1667) loss : 6.478707790374756
e : 5 (1350/1667) loss : 6.139671325683594
input: 史 上 最 有 名 的 小 偷 是 誰 ？ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 阿 扁 啊 戰 哥 說 他 竊 國 

e : 5 (1458/1667) loss : 6.498518466949463
e : 5 (1459/1667) loss : 6.321186542510986
e : 5 (1460/1667) loss : 6.192043781280518
input: 有 沒 有 關 於 抓 耙 仔 的 掛 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 不 會 怎 樣 ， 通 常 活 得 好 好 的 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 
e : 5 (1461/1667) loss : 6.200606346130371
e : 5 (1462/1667) loss : 6.383484840393066
e : 5 (1463/1667) loss : 6.179409503936768
e : 5 (1464/1667) loss : 6.274679660797119
e : 5 (1465/1667) loss : 6.441880226135254
e : 5 (1466/1667) loss : 6.435641288757324
e : 5 (1467/1667) loss : 6.080558776855469
e : 5 (1468/1667) loss : 6.376297950744629
e : 5 (1469/1667) loss : 6.147581577301025
e : 5 (1470/1667) loss : 6.116757869720459
input: 履 歷 忘 了 加 掛 畢 業 證 書 會 不 會 飛 高 高 [PAD] [PAD] [PAD] 
target: 學 店 沒 人 在 意 好 ㄇ 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 。 。 。 。 。 。 。 

e : 5 (1581/1667) loss : 6.121817588806152
e : 5 (1582/1667) loss : 6.38693904876709
e : 5 (1583/1667) loss : 6.147017955780029
e : 5 (1584/1667) loss : 6.282375335693359
e : 5 (1585/1667) loss : 6.456071376800537
e : 5 (1586/1667) loss : 6.360473155975342
e : 5 (1587/1667) loss : 5.990622043609619
e : 5 (1588/1667) loss : 6.248847484588623
e : 5 (1589/1667) loss : 6.256185054779053
e : 5 (1590/1667) loss : 6.429052829742432
input: 有 便 宜 雞 蛋 的 掛 嗎 ？ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 良 率 問 題 便 宜 蛋 打 開 裡 面 混 成 一 團 機 率 頗 高 。 [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 
e : 5 (1591/1667) loss : 6.42868185043335
e : 5 (1592/1667) loss : 6.283736228942871
e : 5 (1593/1667) loss : 6.293147087097168
e : 5 (1594/1667) loss : 6.329301834106445
e : 5 (1595/1667) loss : 6.266907691955566
e : 5 (1596/1667) loss : 6.252986907958984
e : 5 (1597/1667) loss : 6.241881847381592
e : 5 (1598/1667) loss : 6.318767070770264
e : 5 (15

e : 6 (43/1667) loss : 6.281872749328613
e : 6 (44/1667) loss : 6.367873668670654
e : 6 (45/1667) loss : 6.30680513381958
e : 6 (46/1667) loss : 6.505914211273193
e : 6 (47/1667) loss : 6.255163669586182
e : 6 (48/1667) loss : 6.3238139152526855
e : 6 (49/1667) loss : 6.3338212966918945
e : 6 (50/1667) loss : 6.2423319816589355
input: 說 自 己 個 性 大 剌 剌 的 女 孩 在 想 什 麼 [PAD] [PAD] 
target: 暗 示 說 打 就 打 說 幹 就 幹 。 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 
e : 6 (51/1667) loss : 6.334434986114502
e : 6 (52/1667) loss : 6.41023063659668
e : 6 (53/1667) loss : 6.278877258300781
e : 6 (54/1667) loss : 6.217292308807373
e : 6 (55/1667) loss : 6.26838493347168
e : 6 (56/1667) loss : 6.300516128540039
e : 6 (57/1667) loss : 6.349171161651611
e : 6 (58/1667) loss : 6.50049352645874
e : 6 (59/1667) loss : 6.482438087463379
e : 6 (60/1667) loss : 6.119840621948242
input: 台 灣 建 商 為 何 只 蓋 房 子 很 少 加 其 他 機 能 [PAD] [PAD] 
target: 一 堆 

KeyboardInterrupt: 

In [ ]:
test_data = DataLoader(v_set, shuffle=True, batch_size=1, collate_fn=create_mini_batch)

In [ ]:
from torchviz import make_dot, make_dot_from_trace

In [ ]:
# import os
# os.environ["PATH"] += os.pathsep + 'C:\Program Files\Graphviz 2.44.1\bin'
# os.environ["PATH"] += os.pathsep + 'C:\ProgramData\Anaconda3\Lib\site-packages\graphviz'

TRAIN_DIALOG = True
x,y = train_set.__getitem__(0)
make_dot(g_model(x,y,1), params=dict(g_model.named_parameters())).render("transformer", format="png")